# *******************************************************
# IGTI - Bootcamp Analista de Machine Learning
# Módulo 2 - Modelos Preditivos e Séries Temporais
# DESAFIO - GISELLE S C
# *******************************************************

In [ ]:
#importando as bibliotecas
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import datetime
from sklearn.model_selection import train_test_split,KFold,cross_val_score, cross_val_predict # esse método é utilizado para dividir o 
                                                    # conjunto de dados em grupos de treinamento e test
from sklearn.svm import SVC      #importa o algoritmo svm para ser utilizado 
from sklearn import tree         # importa o algoritmo arvore de decisão
from sklearn.linear_model import LogisticRegression #importa o algoritmo de regressão logística
from sklearn.metrics import mean_absolute_error #utilizada para o calculo do MAE
from sklearn.metrics import mean_squared_error #utilizada para o calculo do MSE
from sklearn.metrics import r2_score #utilizada para o calculo do R2
from sklearn import metrics  #utilizada para as métricas de comparação entre os métodos
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score

In [ ]:
#importando o arquivo para o google colab
from google.colab import files
uploaded = files.upload()

Saving winequality-red.csv to winequality-red.csv


In [ ]:
dados_wine=pd.read_csv('winequality-red.csv',sep=";") #carregando os dados

In [ ]:
dados_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
dados_wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [ ]:
dados_wine.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [ ]:
dados_wine.shape

(1599, 12)

In [ ]:
dados_wine.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [ ]:
#Cálculo da correlação para as variáveis selecionadas
dados_corr=dados_wine[['fixed acidity','pH']]
corr=dados_corr.corr(method='pearson')
print(corr)
##corr.style.background_gradient(cmap='coolwarm')

               fixed acidity        pH
fixed acidity       1.000000 -0.682978
pH                 -0.682978  1.000000


In [ ]:
dados_corr=dados_wine[['quality','alcohol']]
corr=dados_corr.corr(method='pearson')
print(corr)

          quality   alcohol
quality  1.000000  0.476166
alcohol  0.476166  1.000000


## *Poderia ter feito assim tb:*

df_vinhos[['fixed acidity', 'pH']].corr()

df_vinhos[['quality', 'alcohol']].corr() 

## *Ou assim: Podemos usar também o método: *

df['fixed acidity'].corr(df['pH']) para análise das 2 variaveis.



## *Outro método:*

import seaborn as sn #biblioteca utilizada pra plotar os gráficos em um formato mais "elegante"
### realizando o plot da matriz de correlação
dtCorrel=dataset[["fixed acidity", "alcohol"]]
plt.figure(figsize=(10, 5))
matriz_de_correlação = dtCorrel.corr()  #construindo a matriz de correlação
sn.heatmap(matriz_de_correlação, annot=True,vmin=-1, vmax=1,center= 0)  #plotando a matriz de correlação com o seaborn
plt.show()


In [ ]:
dados_wine['quality'].value_counts()  # ou colocando df[df["quality"] == 5] no final para ter somente esta categoria

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

**Preparando os dados para as modelagens**

In [ ]:
dados_wine.head()

In [ ]:
#transforma os dados em array
x = dados_wine.iloc[:, :-1].values  #dados de entrada
y = dados_wine.iloc[:, 11].values  # saídas ou target  (quality)

In [ ]:
x.shape,y.shape

((1599, 11), (1599,))

In [ ]:
from sklearn.preprocessing import MinMaxScaler  #aplicando a normalização dos dados
scaler = MinMaxScaler(feature_range=(0, 1)) #cria o objeto que realiza a normalização dos dados por meio dos valores mínimos e máximos

In [ ]:
fa = dados_wine['fixed acidity'].values
fa_norm = scaler.fit_transform(fa.reshape(-1, 1)) # aplica a escala

In [ ]:
min(fa_norm)

array([0.])

## *poderia ter feito desta forma:*

entradas_normalizadas[:,0].min()

In [ ]:
entradas_normalizadas = scaler.fit_transform(x.reshape(-11, 11)) # aplica a escala

O algoritmo KNeighborsClassifier nao solicita que a variavel dependente seja de 2D. Neste caso a variável deve ser de 1D, por isso, nao precisa usar o reshape.
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html


In [ ]:
from sklearn.model_selection import train_test_split   # função que realiza a divisão do dataset
X_train, X_test, y_train, y_test = train_test_split(entradas_normalizadas, y, test_size=0.30, random_state=1)

In [ ]:
X_train.shape,y_train.shape, X_test.shape, y_test.shape

((1119, 11), (1119,), (480, 11), (480,))

**Modelo KNN**

In [ ]:
#importando as bibliotecas 
import pylab as pl # utilizado para plotar os gráficos
from sklearn import neighbors  #biblioteca do KNN e de conjunto de dados (dataset)

In [ ]:
#treina o modelo 
from sklearn.neighbors import KNeighborsClassifier
clf_KNN = KNeighborsClassifier(n_neighbors=5) #utiliza a construção por meio de 5 vizinhos
clf_KNN.fit(X_train, y_train) # aplica a classificação

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
#realiza a previsão com os dados de teste
y_pred_KNN = clf_KNN.predict(X_test) 

In [ ]:
acuracia_KNN = accuracy_score(y_test, y_pred_KNN)
print('Acurácia do KNN: ',acuracia_KNN)

Acurácia do KNN:  0.5645833333333333


**ARVORE DE DECISÃO**

In [ ]:
from sklearn.tree import DecisionTreeClassifier # importa o classificador árvore de decisão
from sklearn import metrics #importa as métricas para avaliaçãon de modelos

In [ ]:
# Cria o objeto de classificação através
clf_AD = DecisionTreeClassifier()

# Realiza o treinamento do classificador
clf_AD = clf_AD.fit(X_train,y_train)

#Realiza a previsão de classificaçao 
y_pred_AD = clf_AD.predict(X_test)

In [ ]:
acuracia_AD = accuracy_score(y_test, y_pred_AD)
print('Acurácia da Árvore de Decisão: ',acuracia_AD)

Acurácia da Árvore de Decisão:  0.5833333333333334


**Random Forest**

In [ ]:
clf_forest = RandomForestClassifier(max_depth=10, random_state=1)
clf_forest = clf_forest.fit(X_train,y_train)

In [ ]:
#Realiza a previsão de classificaçao 
y_pred_forest = clf_forest.predict(X_test)

In [ ]:
acuracia_forest = accuracy_score(y_test, y_pred_forest)
print('Acurácia do Random Forest: ',acuracia_forest)

Acurácia do Random Forest:  0.6791666666666667


**Algoritmo SVM**


In [ ]:
clf_svm=SVC(gamma='auto',kernel='rbf')
clf_svm = clf_svm.fit(X_train,y_train)

#Realiza a previsão de classificaçao 
y_pred_svm = clf_svm.predict(X_test)

In [ ]:
acuracia_svm = accuracy_score(y_test, y_pred_svm)
print('Acurácia do SVM: ',acuracia_svm)

Acurácia do SVM:  0.5854166666666667


**Algoritmo Rede MLP**

In [ ]:
#definição da biblioteca
from sklearn.neural_network import MLPClassifier

clf_mlp = MLPClassifier( alpha=1e-5, hidden_layer_sizes=(5, 5), random_state=1)
clf_mlp = clf_mlp.fit(X_train,y_train)

#Realiza a previsão de classificaçao 
y_pred_mlp = clf_mlp.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
acuracia_mlp = accuracy_score(y_test, y_pred_mlp)
print('Acurácia do MLP: ',acuracia_mlp)

Acurácia do MLP:  0.6


**Alterando a saída para binária e aplicando o Random Forest**

In [ ]:
dados_wine['quality_bin']=dados_wine['quality'].replace({6:1,7:1,8:1,3:0,4:0,5:0})
dados_wine['quality_bin'].value_counts()

1    855
0    744
Name: quality_bin, dtype: int64

In [ ]:
dados_wine.head()

Preparando os dados com a nova saída

In [ ]:
#transforma os dados em array
x = dados_wine.iloc[:, :-2].values  #dados de entrada
y_bin = dados_wine.iloc[:, 12].values  # saídas ou target  (quality)

In [ ]:
x.shape,y_bin.shape

((1599, 11), (1599,))

In [ ]:
from sklearn.preprocessing import MinMaxScaler  #aplicando a normalização dos dados
scaler = MinMaxScaler(feature_range=(0, 1)) #cria o objeto que realiza a normalização dos dados por meio dos valores mínimos e máximos

In [ ]:
entradas_normalizadas = scaler.fit_transform(x.reshape(-11, 11)) # aplica a escala

In [ ]:
from sklearn.model_selection import train_test_split   # função que realiza a divisão do dataset
X_train, X_test, y_bin_train, y_bin_test = train_test_split(entradas_normalizadas, y_bin, test_size=0.30, random_state=1)

In [ ]:
X_train.shape,y_bin_train.shape, X_test.shape, y_bin_test.shape

((1119, 11), (1119,), (480, 11), (480,))

**Random Forest com saída binária**

In [ ]:
clf_forest_bin = RandomForestClassifier(max_depth=10, random_state=1)
clf_forest_bin = clf_forest_bin.fit(X_train,y_bin_train)

In [ ]:
#Realiza a previsão de classificaçao 
y_bin_pred_forest = clf_forest_bin.predict(X_test)

In [ ]:
acuracia_forest_bin = accuracy_score(y_bin_test, y_bin_pred_forest)
print('Acurácia do Random Forest (saída binária): ',acuracia_forest_bin)

Acurácia do Random Forest (saída binária):  0.8041666666666667
